In [138]:
import os
import re
import numpy as np
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv

In [139]:
# %pip install mysql-connector-python
# %pip install python-dotenv

### 0. Data Loading

In [3]:
def getYearDirs(data_dir, year_pat="\d{4}"):
    """
    Get year subfolder directories of a main data directory.
    Regex matches folder in year format (i.e., 4 digits).
    """
    subfolders = [f.name for f in os.scandir(data_dir) if (f.is_dir() and bool(re.match(year_pat, f.name)))] # Get all subfolders
    subfolder_paths = [os.path.join(data_dir, f) for f in subfolders] # Create path
    return subfolder_paths

def getQuarterDatDir(year_dir, fname_pat="^metro\-trips\-\d{4}\-q\d"):
    """
    Get quarter data file path from a folder of year.
    """
    files = [f.name for f in os.scandir(year_dir) if (f.is_file() and bool(re.match(fname_pat, f.name)))]
    file_paths = [os.path.join(year_dir, f) for f in files] # Create path
    return file_paths

def getAllData(data_dir):
    fpaths = list()
    for year in getYearDirs(data_dir):
        for path in getQuarterDatDir(year):
            fpaths.append(path)
    return fpaths

In [4]:
### List all the files in the directories
data_dir = ".\\data\\"
files_dir = getAllData(data_dir)

In [5]:
for d in files_dir:
    print(d)

.\data\2022\metro-trips-2022-q1.csv
.\data\2022\metro-trips-2022-q2.csv
.\data\2022\metro-trips-2022-q3.csv
.\data\2022\metro-trips-2022-q4.csv
.\data\2023\metro-trips-2023-q1.csv
.\data\2023\metro-trips-2023-q2.csv
.\data\2023\metro-trips-2023-q3.csv
.\data\2023\metro-trips-2023-q4.csv
.\data\2024\metro-trips-2024-q1.csv
.\data\2024\metro-trips-2024-q2.csv
.\data\2024\metro-trips-2024-q3.csv
.\data\2024\metro-trips-2024-q4.csv


### 1. Dataset Analysis from 2022 to 2024

#### 1.1 Station Analysis

In [6]:
station_path = ".\data\stations\metro-bike-share-stations-2024-10-01.csv"
station_df = pd.read_csv(station_path, sep=",",encoding="cp1252")

In [7]:
station_df.head(5)

,Kiosk ID,Kiosk Name,Go Live Date,Region,Status,Latitude,Longitude
0,3000.0,Virtual Station,7/7/2016,NaN,Active,0.000000,0.000000
1,3005.0,7th & Flower,7/7/2016,DTLA,Active,34.048500,-118.258537
2,3006.0,Olive & 8th,7/7/2016,DTLA,Active,34.045540,-118.256668
3,3007.0,5th & Grand,7/7/2016,DTLA,Active,34.050480,-118.254593
4,3008.0,Figueroa & 9th,7/7/2016,DTLA,Active,34.046612,-118.262733


In [8]:
station_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Kiosk ID      423 non-null    float64
 1   Kiosk Name    423 non-null    object 
 2   Go Live Date  423 non-null    object 
 3   Region        421 non-null    object 
 4   Status        423 non-null    object 
 5   Latitude      423 non-null    float64
 6   Longitude     423 non-null    float64
dtypes: float64(3), object(4)
memory usage: 23.5+ KB


In [9]:
station_df[station_df.isnull().any(axis=1)]

,Kiosk ID,Kiosk Name,Go Live Date,Region,Status,Latitude,Longitude
0,3000.0,Virtual Station,7/7/2016,NaN,Active,0.00000,0.00000
421,4682.0,CicLAvia Meet the Hollywood's - West Hollywood...,8/18/2024,NaN,Active,34.08564,-118.38279
423,NaN,NaN,NaN,NaN,NaN,NaN,NaN
424,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
station_df = station_df.dropna(how="all")

In [11]:
station_df[station_df.isnull().any(axis=1)]

,Kiosk ID,Kiosk Name,Go Live Date,Region,Status,Latitude,Longitude
0,3000.0,Virtual Station,7/7/2016,NaN,Active,0.00000,0.00000
421,4682.0,CicLAvia Meet the Hollywood's - West Hollywood...,8/18/2024,NaN,Active,34.08564,-118.38279


In [12]:
## Due to encoding there are some inconsistency in the naming
station_df.columns = [col_name.strip().lower().replace(" ", "_") for col_name in station_df.columns]

In [13]:
print(station_df.columns)

Index(['kiosk_id', 'kiosk_name', 'go_live_date', 'region', 'status',
       'latitude', 'longitude'],
      dtype='object')


In [14]:
station_df["region"].unique()

array([nan, 'DTLA', 'Port of LA', 'Pasadena', 'Westside', 'Free Bikes',
       'North Hollywood'], dtype=object)

In [15]:
station_df["region"] = station_df["region"].fillna("Not Available")
print(station_df["region"].unique())

['Not Available' 'DTLA' 'Port of LA' 'Pasadena' 'Westside' 'Free Bikes'
 'North Hollywood']


**Summary**<br>
- Station data requires other ecoding method from the widely used UTF-8.
- Certain station region do not have available region. We will fill it with 'Not Available'.

#### 1.2 Bike Data Analysis

In [16]:
def format_bike_id(id):
    """
    Bike ID is normally an integer but in some cases they could contain special characters or a text.
    """
    if id.isdigit():
        return int(id)
    return id

In [17]:
unique_bike_df = pd.DataFrame(columns=["year", "quarter", "bike_id", "bike_type"])
unique_bholder_df = pd.DataFrame(columns=["passholder_type"])

## Loop through all files and append it to unique_bike_df
for file in files_dir:
    ## Data Tracker
    quarter, year = file.split(".")[1].split("-")[:1:-1]
    
    ## Read file
    file_df = pd.read_csv(file, sep=",", low_memory=False) # All at once

    ## 1. Bike Information
    ## Grab Bike info
    bike_df = file_df[["bike_id", "bike_type"]].copy()
    bike_df["year"] = year
    bike_df["quarter"] = quarter

    ## Filter to remove duplicates
    bike_df = bike_df.drop_duplicates(subset=["bike_id", "bike_type"]) # Assume that "bike_id" and "bike_type" are both unique

    ## Add it to the list of unique bikes
    unique_bike_df = pd.concat([unique_bike_df, bike_df], ignore_index=True)
    unique_bike_df["bike_id"] = unique_bike_df["bike_id"].astype(str).apply(format_bike_id)
    unique_bike_df["bike_type"] = unique_bike_df["bike_type"].astype(str)

    ## Sorted and Drop Duplicates
    unique_bike_df.sort_values(by=["bike_id", "bike_type"], inplace=True)
    unique_bike_df.drop_duplicates(subset=["bike_id", "bike_type"], inplace=True)

    ## 2. Passholder Type
    passholder_df = file_df[["passholder_type"]].copy()
    passholder_df.drop_duplicates(subset=["passholder_type"])
    unique_bholder_df = pd.concat([unique_bholder_df, passholder_df], ignore_index=True)
    unique_bholder_df.drop_duplicates(subset=["passholder_type"], inplace=True)

In [18]:
unique_bike_df.reset_index(drop=True)
display(unique_bike_df)

,year,quarter,bike_id,bike_type
0,2022,q2,1349,standard
1,2022,q1,4727,standard
2,2022,q1,4823,standard
3,2022,q1,5006,standard
4,2022,q1,5016,standard
...,...,...,...,...
2335,2023,q1,15450a,smart
2336,2022,q1,15902a,smart
2337,2022,q1,16116a,smart
2338,2022,q1,16257a,smart


In [19]:
unique_bike_df["bike_type"].unique()

array(['standard', 'electric', 'smart'], dtype=object)

In [20]:
df_filtered = unique_bike_df[unique_bike_df["bike_id"].str.contains(r"[a-zA-Z]", regex=True, na=False)]

In [21]:
print(df_filtered)

      year quarter         bike_id bike_type
2334  2023      q4          15316a     smart
2335  2023      q1          15450a     smart
2336  2022      q1          15902a     smart
2337  2022      q1          16116a     smart
2338  2022      q1          16257a     smart
2339  2023      q3  TBlocker_Slash  standard


In [22]:
df_filtered_2 = unique_bike_df[unique_bike_df["bike_type"]=="smart"]
display(df_filtered_2)

,year,quarter,bike_id,bike_type
2334,2023,q4,15316a,smart
2335,2023,q1,15450a,smart
2336,2022,q1,15902a,smart
2337,2022,q1,16116a,smart
2338,2022,q1,16257a,smart


**Summary** <br>
- Bike identifier contains both integer, a mix, and characters.
- As for bike type there are only three types: standard, electric, and smart. Only bike with mized id, such as 15316a, is a smart bike.
- One unique bike is 'TBlocker_Slash', which upon inspection in the original dataset represents testing run for the bike.

#### 1.3 Trips

In [23]:
unique_bholder_df

,passholder_type
0,One Day Pass
1,Walk-up
2,Annual Pass
3,Monthly Pass
4,NaN
5,Testing


### 2. My Database Schema Planning

#### 2.1 Station Table

In [24]:
"""
CREATE TABLE IF NOT EXISTS stations(
    station_id INT,
    name VARCHAR(512) NOT NULL,
    live_date DATE,
    region ENUM('DTLA', 'Port of LA', 'Pasadena', 'Westside', 'Free Bikes', 'North Hollywood', 'Not Available') DEFAULT('Not Available'),
    status ENUM('Active', 'Inactive') NOT NULL,
    latitude DECIMAL(10,7),
    longitude DECIMAL (10,7),
    PRIMARY KEY (station_id)
);
"""

"\nCREATE TABLE IF NOT EXISTS stations(\n    station_id INT,\n    name VARCHAR(512) NOT NULL,\n    live_date DATE,\n    region ENUM('DTLA', 'Port of LA', 'Pasadena', 'Westside', 'Free Bikes', 'North Hollywood', 'Not Available') DEFAULT('Not Available'),\n    status ENUM('Active', 'Inactive') NOT NULL,\n    latitude DECIMAL(10,7),\n    longitude DECIMAL (10,7),\n    PRIMARY KEY (station_id)\n);\n"

#### 2.2 Bike Table

In [25]:
"""
CREATE TABLE IF NOT EXISTS bikes(
    bike_id VARCHAR(128),
    bike_type ENUM('standard', 'electric', 'smart') NOT NULL DEFAULT('standard'),
    PRIMARY KEY (bike_id)
);
"""

"\nCREATE TABLE IF NOT EXISTS bikes(\n    bike_id VARCHAR(128),\n    bike_type ENUM('standard', 'electric', 'smart') NOT NULL DEFAULT('standard'),\n    PRIMARY KEY (bike_id)\n);\n"

#### 2.3 Trips Table

In [26]:
"""
CREATE TABLE IF NOT EXISTS trips(
    trip_id INT,
    bike_id VARCHAR(128) NOT NULL,
    start_station INT,
    end_station INT,
    start_time DATETIME,
    end_time DATETIME,
    duration INT,
    plan_duration INT CHECK (plan_duration >= 0),
    trip_route_category ENUM('Round Trip', 'One Way') NOT NULL,
    passholder_type ENUM('Walk-up', 'One Day Pass', 'Monthly Pass', 'Annual Pass', 'Testing', 'Not Available')  DEFAULT('Not Available'),
    PRIMARY KEY (trip_id),
    FOREIGN KEY (bike_id) REFERENCES bikes(bike_id),
    FOREIGN KEY (start_station) REFERENCES stations(station_id),
    FOREIGN KEY (end_station) REFERENCES stations(station_id)
);
"""

"\nCREATE TABLE IF NOT EXISTS trips(\n    trip_id INT,\n    bike_id VARCHAR(128) NOT NULL,\n    start_station INT,\n    end_station INT,\n    start_time DATETIME,\n    end_time DATETIME,\n    duration INT,\n    plan_duration INT CHECK (plan_duration >= 0),\n    trip_route_category ENUM('Round Trip', 'One Way') NOT NULL,\n    passholder_type ENUM('Walk-up', 'One Day Pass', 'Monthly Pass', 'Annual Pass', 'Testing', 'Not Available')  DEFAULT('Not Available'),\n    PRIMARY KEY (trip_id),\n    FOREIGN KEY (bike_id) REFERENCES bikes(bike_id),\n    FOREIGN KEY (start_station) REFERENCES stations(station_id),\n    FOREIGN KEY (end_station) REFERENCES stations(station_id)\n);\n"

#### 2.4 Loaded Data Tracker

In [27]:
"""
CREATE TABLE IF NOT EXISTS load_history(
    id IT AUTO_INCREMENT,
    file_name VARCHAR(256),
    load_stamp DATETIME DEFAULT CURRENT_TIMESTAMP(),
    row_count INT,
    load_status ENUM('Success', 'Failure') DEFAULT 'Success',
    PRIMARY KEY (id)
);
"""

"\nCREATE TABLE IF NOT EXISTS load_history(\n    id IT AUTO_INCREMENT,\n    file_name VARCHAR(256),\n    load_stamp DATETIME DEFAULT CURRENT_TIMESTAMP(),\n    row_count INT,\n    load_status ENUM('Success', 'Failure') DEFAULT 'Success',\n    PRIMARY KEY (id)\n);\n"

### 3. Data loading

In [132]:
def create_tables(conn):
    """
    Create table for stations, bikes, trips, and load_history.
    """
    ### Define Table
    load_history_table_schema = """
    CREATE TABLE IF NOT EXISTS load_history(
        id INT AUTO_INCREMENT,
        file_name VARCHAR(256),
        load_stamp DATETIME DEFAULT CURRENT_TIMESTAMP(),
        row_count INT,
        load_status ENUM('Success', 'Failure') DEFAULT 'Success',
        PRIMARY KEY (id)
    );
    """
    station_table_schema = """
    CREATE TABLE IF NOT EXISTS stations(
        station_id INT,
        name VARCHAR(512) NOT NULL,
        live_date DATE,
        region ENUM('DTLA', 'Port of LA', 'Pasadena', 'Westside', 'Free Bikes', 'North Hollywood', 'Not Available') DEFAULT('Not Available'),
        status ENUM('Active', 'Inactive') NOT NULL,
        latitude DECIMAL(10,7),
        longitude DECIMAL (10,7),
        PRIMARY KEY (station_id)
    );
    """
    bike_table_schema = """
    CREATE TABLE IF NOT EXISTS bikes(
        bike_id VARCHAR(128),
        bike_type ENUM('standard', 'electric', 'smart') NOT NULL DEFAULT('standard'),
        PRIMARY KEY (bike_id)
    );
    """
    trips_table_schema = """
    CREATE TABLE IF NOT EXISTS trips(
        trip_id INT,
        bike_id VARCHAR(128) NOT NULL,
        start_station INT,
        end_station INT,
        start_time DATETIME,
        end_time DATETIME,
        duration INT,
        plan_duration INT CHECK (plan_duration >= 0),
        trip_route_category ENUM('Round Trip', 'One Way') NOT NULL,
        passholder_type ENUM('Walk-up', 'One Day Pass', 'Monthly Pass', 'Annual Pass', 'Testing', 'Not Available')  DEFAULT('Not Available'),
        PRIMARY KEY (trip_id),
        FOREIGN KEY (bike_id) REFERENCES bikes(bike_id),
        FOREIGN KEY (start_station) REFERENCES stations(station_id),
        FOREIGN KEY (end_station) REFERENCES stations(station_id)
    );
    """
    ### Connect and create tables
    db_cursor = conn.cursor()
    db_cursor.execute(load_history_table_schema)
    db_cursor.execute(station_table_schema)
    db_cursor.execute(bike_table_schema)
    db_cursor.execute(trips_table_schema)
    conn.commit()
    db_cursor.close()

def update_load_history(conn, fname, nrows):
    insert_file_name_sql = """
    INSERT IGNORE INTO load_history (file_name, load_stamp, row_count, load_status)
    VALUES (%s, %s, %s, %s);
    """
    ## Define Values
    db_cursor = conn.cursor()
    dt_now = datetime.now()
    load_stamp = dt_now.strftime("%Y-%m-%d %H:%M:%S")

    ## Add
    db_cursor.execute(
        insert_file_name_sql,
        (fname, load_stamp, nrows, 'Success')
    )
    
    conn.commit()
    db_cursor.close()
    

def insert_stations_data(conn, df):
    insert_station_sql = """
    INSERT IGNORE INTO stations (station_id, name, live_date, region, status, latitude, longitude) 
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    ## Define Values
    station_records = []
    clone_df = df.copy()
    print("Connection Status:", conn.is_connected())
    db_cursor = conn.cursor()

    ## Get all the stations data
    for idx, record in clone_df.iterrows():
        station_id = int(record["station_id"]) if record.get("station_id") else None
        name = str(record.get("name", None))
        live_date = record.get("live_date" ,None)
        region = str(record.get("region", "Not Available"))
        status = str(record["status"]) if record.get("status") else "Active"
        latitude = float(record["latitude"]) if record.get("latitude") else None
        longitude = float(record["longitude"]) if record.get("longitude") else None

        ## Reformat date
        dt_obj = pd.to_datetime(live_date, format="%m/%d/%Y")
        live_date = dt_obj.strftime("%Y-%m-%d %H:%M:%S") # String

        ## Update record
        station_records.append(
            (station_id, name, live_date, region, status, latitude, longitude)
        )

    ## Insert Combined Data into the Table: https://dev.mysql.com/doc/connectors/en/connector-python-api-mysqlcursor-executemany.html
    db_cursor.executemany(
        insert_station_sql,
        station_records
    )
    conn.commit()
    db_cursor.close()

def insert_bikes_data(conn, df):
    insert_bike_sql = """
    INSERT IGNORE INTO bikes (bike_id, bike_type)
    VALUES (%s, %s)
    """

    ## Define Values
    bike_records = []
    clone_df = df.copy()
    db_cursor = conn.cursor()

    ## Bikes Data
    for idx, record in clone_df.iterrows():
        bike_id = str(record["bike_id"]).strip()
        bike_type = str(record.get("bike_type", None))

        ## Update record
        bike_records.append(
            (bike_id, bike_type)
        )

    ## Insert into table
    db_cursor.executemany(
        insert_bike_sql,
        bike_records
    )
    conn.commit()
    db_cursor.close()

def get_stations(conn):
    db_cursor = conn.cursor()
    ## Get a list of all stations from DB
    db_cursor.execute(
        """
        SELECT station_id
        FROM stations;
        """
    )
    stations = [row[0] for row in db_cursor.fetchall()]
    db_cursor.close()
    return stations

def validate_station(stations, station):
    if station in stations:
        return True
    else:
        return False

def insert_trips_data(conn, df):
    insert_trip_sql = """
    INSERT INTO trips (trip_id, bike_id, start_station, end_station, start_time, end_time, duration, plan_duration, trip_route_category, passholder_type)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    ## Define Values
    trip_records = []
    clone_df = df.copy()
    db_cursor = conn.cursor()

    ## Get stations name for validation
    stations = get_stations(db_conn)

    ## Trips Data
    for idx, record in clone_df.iterrows():
        trip_id = int(record["trip_id"])
        bike_id = str(format_bike_id(str(record["bike_id"]))) # string 
        start_station = int(record["start_station"]) if record.get("start_station") else None
        end_station = int(record["end_station"]) if record.get("end_station") else None
        start_time = record.get("start_time", None)
        end_time = record.get("end_time", None)
        duration = int(record["duration"]) if record.get("duration") else 0
        plan_duration = int(record["plan_duration"]) if record.get("plan_duration") else 0
        trip_route_category = str(record.get("trip_route_category", "One Way"))
        passholder_type = record.get("passholder_type") or "Not Available"

        ## Reformat date
        start_dt_obj = pd.to_datetime(start_time, format="%m/%d/%Y %H:%M")
        start_time = start_dt_obj.strftime("%Y-%m-%d %H:%M:%S") # String
        end_dt_obj = pd.to_datetime(end_time, format="%m/%d/%Y %H:%M")
        end_time = end_dt_obj.strftime("%Y-%m-%d %H:%M:%S") # String

        ## Skip record if it is not in stations
        if validate_station(stations, start_station) and validate_station(stations, end_station):
            trip_records.append(
                (trip_id, bike_id, start_station, end_station, start_time, end_time, duration, plan_duration, trip_route_category, passholder_type)
            )

    ## Insert into table
    db_cursor.executemany(
        insert_trip_sql,
        trip_records
    )
    conn.commit()
    db_cursor.close()

In [133]:
def check_loaded_status(conn, fname):
    """
    Function to check if the file has already been loaded or not.
    If not it will log the file name into the history table and return False.
    """
    db_cursor = conn.cursor()

    ## Get the list of filenames
    db_cursor.execute(
        """
        SELECT file_name
        FROM load_history
        """
    )
    entries = db_cursor.fetchall() # tuple list
    loaded_fnames = [entry[0] for entry in entries] # get first element of tuple in a list
    if (not fname) or (fname not in loaded_fnames):
        db_cursor.close()
        ## Loaded = False
        return False
    else:
        print(f"This {fname} file has already loaded in this database.")
        db_cursor.close()
        
        ## Loaded = True
        return True 

In [143]:
load_dotenv()
MySQL_SECRET = os.getenv("MySQL_ROOT_PASS")

In [144]:
import mysql.connector as sql

### Connect to MySQL
db_conn = sql.connect(
    host="localhost",
    user="root",
    password=MySQL_SECRET,
    use_pure=True
)

print("Connected:", db_conn.is_connected())

Connected: True


In [145]:
cursor = db_conn.cursor()
cursor.execute("SHOW DATABASES;")
databases = cursor.fetchall()
print("Databases on this MySQL Server:")
for db in databases:
    print(db[0])

Databases on this MySQL Server:
bike_share
information_schema
mysql
performance_schema
sakila
sys
world


In [103]:
cursor.execute(
    """
    DROP DATABASE IF EXISTS bike_share;
    """
)

In [117]:
cursor.execute(
    """
    CREATE DATABASE IF NOT EXISTS bike_share;
    """
)
cursor.execute(
    """
    USE bike_share;
    """
)

In [118]:
cursor.execute("SELECT DATABASE();")
current_db = cursor.fetchone()
print("Current database is:", current_db)

Current database is: ('bike_share',)


In [119]:
create_tables(db_conn) # create tables

In [107]:
### Format Station Dataframe
station_df.head(5)
station_df = station_df.rename(
    columns={
        "kiosk_id": "station_id", 
        "kiosk_name": "name",
        "go_live_date": "live_date",
        "region": "region",
        "status": "status",
        "latitude": "latitude",
        "longitude": "longitude"
    }
)

In [108]:
### Load Stations Table
insert_stations_data(db_conn, station_df)

Connection Status: True


In [109]:
### Format bike data
formatted_bike_df = unique_bike_df[["bike_id", "bike_type"]]
formatted_bike_df

,bike_id,bike_type
0,1349,standard
1,4727,standard
2,4823,standard
3,5006,standard
4,5016,standard
...,...,...
2335,15450a,smart
2336,15902a,smart
2337,16116a,smart
2338,16257a,smart


In [110]:
### load Bike ID Data
insert_bikes_data(db_conn, formatted_bike_df)

In [134]:
## Sort Order of loading
for file in files_dir:

    ## Check if file is already loaded pass the file
    fname = file.split("\\")[3].split(".")[0]
    if check_loaded_status(db_conn, fname) == True:
        print(f"File {fname} is skipped.")
        continue

    ## Load and insert
    file_df = pd.read_csv(file, sep=",", low_memory=False)
    nrows = file_df.shape[0]
    insert_trips_data(db_conn, file_df)

    ## Update loaded files
    update_load_history(db_conn, fname, nrows)

    ## Update Status
    print(f"File {fname} is loaded.")

This metro-trips-2022-q1 file has already loaded in this database.
File metro-trips-2022-q1 is skipped.
This metro-trips-2022-q2 file has already loaded in this database.
File metro-trips-2022-q2 is skipped.
This metro-trips-2022-q3 file has already loaded in this database.
File metro-trips-2022-q3 is skipped.
This metro-trips-2022-q4 file has already loaded in this database.
File metro-trips-2022-q4 is skipped.
This metro-trips-2023-q1 file has already loaded in this database.
File metro-trips-2023-q1 is skipped.
This metro-trips-2023-q2 file has already loaded in this database.
File metro-trips-2023-q2 is skipped.
This metro-trips-2023-q3 file has already loaded in this database.
File metro-trips-2023-q3 is skipped.
This metro-trips-2023-q4 file has already loaded in this database.
File metro-trips-2023-q4 is skipped.
This metro-trips-2024-q1 file has already loaded in this database.
File metro-trips-2024-q1 is skipped.
This metro-trips-2024-q2 file has already loaded in this databas

In [146]:
db_conn.close()

### 4. Debug Zone

In [126]:
### Debug Key error
file_df = pd.read_csv(files_dir[-1], sep=",", low_memory=False)
q4_2024 = file_df["end_station"].astype("int").unique()

In [130]:
all_stations = list(station_df["station_id"].astype("int").unique())

In [131]:
result = [item for item in q4_2024 if item not in all_stations]
print(result)

[4686, 4687, 4689]
